<a href="https://colab.research.google.com/github/jooyeongkang/kaggle-chest-x-ray-images-pneumonia/blob/master/ChestXRay_Pneumonia_4.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import libraries

In [ ]:
! pip install --upgrade pip &> /dev/null
! pip install tensorflow &> /dev/null
! pip install -q kaggle &> /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##from google.colab import files
#uploaded = files.upload()

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import seaborn as sns
from sklearn.metrics import f1_score
from PIL import Image

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Load data

## Jooyeong's Path

In [ ]:
# Train Data
dir_dt_train = '/content/drive/MyDrive/Data Science/Datasets/chest_xray/train'
dir_dt_train = pathlib.Path(dir_dt_train)

# Validation Data(merged to the train data due to the small amount of validation data)
#dir_dt_val = '/content/drive/MyDrive/Data Science/Datasets/chest_xray/val'
#dir_dt_val = pathlib.Path(dir_dt_val)

# Test Data
dir_dt_test = '/content/drive/MyDrive/Data Science/Datasets/chest_xray/test'
dir_dt_test = pathlib.Path(dir_dt_test)

# Create a dataset

## Define parameters

In [ ]:
batch_size = 32   # Number of Images in each batch
img_height = 180  # Size of Height for Resizing
img_width = 180   # Size of Width for Resizing

## Data Preprocessing

### Reference
**tf.keras.preprocessing.image_dataset_from_directory**
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory

### Subset of Original Data
#### Train: 200 images
#### Validation: 100 images
#### Test: 624 images

In [ ]:
dt_train = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir_dt_train,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='grayscale', # Option: rgb
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split= 1-(200/5232),
    subset='training')

# Number of category
num_class = len(dt_train.class_names)

In [ ]:
dt_val = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir_dt_train,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='grayscale', # Option: rgb
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=100/5232,
    subset='validation')

In [ ]:
dt_test = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir_dt_test,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='grayscale', # Option: rgb
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123)

## Check Data Structure

### Reference
**tf.data.Dataset**

https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [ ]:
for image_batch, labels_batch in dt_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# Visualize Data

In [ ]:
def visualize_img(dt_aug):

  plt.figure(figsize=(10, 10))

  if dt_aug != '': # Augmented images
    if dt_aug == 'randomflip':
      print('RandomFlip Images')
      dt_augm_filp = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal")])
      for images, labels in dt_train.take(1):
        for i in range(9):
          augmented_images = dt_augm_filp(images)
          ax = plt.subplot(3, 3, i + 1)
          plt.imshow(tf.squeeze(augmented_images[i].numpy().astype("uint8")), cmap='gray', vmin=0, vmax=255)
          plt.title(dt_train.class_names[labels[i]])
          plt.axis("off")
          
    elif dt_aug == 'randomrotation':
      print('RandomRotation Images')
      dt_augm_rotation = tf.keras.Sequential([layers.experimental.preprocessing.RandomRotation(0.01)])
      for images, labels in dt_train.take(1):
        for i in range(9):
          augmented_images = dt_augm_rotation(images)
          ax = plt.subplot(3, 3, i + 1)
          plt.imshow(tf.squeeze(augmented_images[i].numpy().astype("uint8")), cmap='gray', vmin=0, vmax=255)
          plt.title(dt_train.class_names[labels[i]])
          plt.axis("off")

    elif dt_aug == 'randomzoom':
      print('RandomZoom Images')
      dt_augm_zoom = tf.keras.Sequential([layers.experimental.preprocessing.RandomZoom(0.1)])
      for images, labels in dt_train.take(1):
        for i in range(9):
          augmented_images = dt_augm_zoom(images)
          ax = plt.subplot(3, 3, i + 1)
          plt.imshow(tf.squeeze(augmented_images[i].numpy().astype("uint8")), cmap='gray', vmin=0, vmax=255)
          plt.title(dt_train.class_names[labels[i]])
          plt.axis("off")

  else: # Original images
    print('Original Images')
    for images, labels in dt_train.take(1):
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(tf.squeeze(images[i].numpy().astype("uint8")), cmap='gray', vmin=0, vmax=255)
        plt.title(dt_train.class_names[labels[i]])
        plt.axis("off")

## Example of Original Images

In [ ]:
visualize_img('')

## Example of Augmented Images

### RandomFlip

In [ ]:
visualize_img('randomflip')

### RandomRotation

In [ ]:
visualize_img('randomrotation')

### RandomZoom

In [ ]:
visualize_img('randomzoom')

# Configure the dataset for the better performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

dt_train = dt_train.cache().prefetch(buffer_size=AUTOTUNE)
dt_val = dt_val.cache().prefetch(buffer_size=AUTOTUNE)

# Experimental Designs

## **Define functions to design models**



1.   Non-Contrastive Learning
  *   Dropout
2.   Contrastive Learning
  *   Data Augmentation (RandomFlip, RandomRotation, RandomZoom)
  *   Dropout

In [ ]:
'''
Hyperparameters
'''
# Number of epochs
epochs = 100

# Dictionary to store mean accuracies per model
mean_accuracies_models = {}

def build_model(dt_train, dt_augms):

  model = Sequential()
  
  '''
  It is also possible to normalize layer in the building block of the model                   
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  The first convolutional layer can typically have a large kernel, usually with a stride of 2
  '''

  if dt_augms == []:
    print('Non-Contrastive Learning')
    model.add(layers.Conv2D(filters=16, kernel_size=5, strides=2, padding='same', activation='relu', input_shape=(img_height, img_width, 1)))
  else:
    print('Contrastive Learning')
    if dt_augms == {'randomflip'}:
      print('RandomFlip')
      model.add(layers.experimental.preprocessing.RandomFlip("horizontal", seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomrotation'}:
      print('RandomRotation')
      model.add(layers.experimental.preprocessing.RandomRotation(0.01, seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomzoom'}:
      print('RandomZoom')
      model.add(layers.experimental.preprocessing.RandomZoom(0.05, seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomflip', 'randomrotation'}:
      print('RandomFlip & RandomRotation')
      model.add(layers.experimental.preprocessing.RandomFlip("horizontal", seed=123, input_shape=(img_height, img_width, 1)))
      model.add(layers.experimental.preprocessing.RandomRotation(0.01, seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomflip', 'randomzoom'}:
      print('RandomFlip & RandomZoom')
      model.add(layers.experimental.preprocessing.RandomFlip("horizontal", seed=123, input_shape=(img_height, img_width, 1)))
      model.add(layers.experimental.preprocessing.RandomZoom(0.05, seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomrotation', 'randomzoom'}:
      print('RandomRotation & RandomZoom')
      model.add(layers.experimental.preprocessing.RandomRotation(0.01, seed=123, input_shape=(img_height, img_width, 1)))
      model.add(layers.experimental.preprocessing.RandomZoom(0.05, seed=123, input_shape=(img_height, img_width, 1)))
    elif dt_augms == {'randomflip', 'randomrotation', 'randomzoom'}:
      print('RandomFlip & RandomRotation & RandomZoom')
      model.add(layers.experimental.preprocessing.RandomFlip("horizontal", seed=123, input_shape=(img_height, img_width, 1)))
      model.add(layers.experimental.preprocessing.RandomRotation(0.01, seed=123, input_shape=(img_height, img_width, 1)))
      model.add(layers.experimental.preprocessing.RandomZoom(0.05, seed=123, input_shape=(img_height, img_width, 1)))

    model.add(layers.Conv2D(filters=16, kernel_size=5, strides=2, padding='same', activation='relu'))
       
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
  model.add(layers.MaxPooling2D())
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(.3))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(.3))
  model.add(layers.Dense(num_class, activation='softmax'))

  # Compile the model
  model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  
  return model


def train_model(model):
  
  # Callback to save the best model
  checkpoint_filepath = '/tmp/checkpoint'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)

  # Train the model
  history = model.fit(
      x=dt_train,
      validation_data=dt_val,
      epochs=epochs,
      callbacks=[model_checkpoint_callback],
      verbose=0)
  
  return history


def plot_evaluation(history):

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(len(history.history['val_accuracy']))

  plt.figure(figsize=(12, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()


def test_model(observed_accuracies):
  # Load the best model through all epochs from the file path
  optimized_model = tf.keras.models.load_model('/tmp/checkpoint')

  # Test and evaluate the model
  val_score = optimized_model.evaluate(dt_val, batch_size=batch_size, verbose=0)
  #print("val_score", val_score[1])
  test_score = optimized_model.evaluate(dt_test, batch_size=batch_size, verbose=0)
  #print("test_score", test_score[1])

  observed_accuracies = np.append(observed_accuracies, test_score[1])
  #print(observed_accuracies)
  return observed_accuracies

## **Convolutional Neural Network(CNN): Non-Contrastive Learning Approach**

##### Build Model

In [ ]:
dt_augms = []
model_cnn_non_contrastive = build_model(dt_train, dt_augms)
model_cnn_non_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_non_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['non-constractive'] = mean_accuracies

## **Convolutional Neural Network(CNN): Contrastive Learning Approach**

### 1.Apply one augmentation:

#### 1-1 RandomFlip

##### Build Model

In [ ]:
dt_augms = {'randomflip'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
#print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-flip'] = mean_accuracies

#### 1-2 RandomRotation

##### Build Model

In [ ]:
dt_augms = {'randomrotation'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-rotate'] = mean_accuracies

#### 1-3 RandomZoom 

##### Build Model

In [ ]:
dt_augms = {'randomzoom'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-zoom'] = mean_accuracies

### 2.Apply two augmentations

#### 2-1 RandomFlip & RandomRotation

##### Build Model

In [ ]:
dt_augms = {'randomflip', 'randomrotation'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-flip-rotate'] = mean_accuracies

#### 2-2 RandomFlip & RandomZoom

##### Build Model

In [ ]:
dt_augms = {'randomflip', 'randomzoom'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-flip-zoom'] = mean_accuracies

#### 2-3 RandomRotation & RotationZoom

##### Build Model

In [ ]:
dt_augms = {'randomrotation', 'randomzoom'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-rotate-zoom'] = mean_accuracies

### 3.Apply all three augmentations

#### 3-1 RandomFlip, RandomRotation & RandomZoom

##### Build Model

In [ ]:
dt_augms = {'randomflip', 'randomrotation', 'randomzoom'}
model_cnn_contrastive = build_model(dt_train, dt_augms)
model_cnn_contrastive.summary()

##### Train & Test & Evaluate Model

In [ ]:
observed_accuracies = np.array([])

for i in range(100):

  history = train_model(model_cnn_contrastive)

  if i == 0: # Sample
    plot_evaluation(history)

  observed_accuracies = test_model(observed_accuracies)

mean_accuracies = observed_accuracies.mean()
print("Mean Accuracies:", mean_accuracies)
mean_accuracies_models['constractive-random-flip-rotate-zoom'] = mean_accuracies

# Compare Average Accuracies among Models 

In [ ]:
for model, avg in mean_accuracies_models.items():
  print("Model: {} | Average accuracy: {}".format(model, avg))